### Bag-of-words classifier

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('file:rozetka_mobile_reviews.json')
df.groupby(['Rating']).count()

,Comment
Rating,
1,315
2,210
3,369
4,1294
5,3837


Розділимо усі рейтинги на два класи:
- _позитивні_ - 5, 4
- _негативні_ - 3, 2, 1

In [3]:
df['Sentiment'] = df['Rating'].apply(lambda x: 'Positive' if x > 3 else 'Negative')
df.groupby(['Sentiment']).count()

,Comment,Rating
Sentiment,,
Negative,894,894
Positive,5131,5131


Подвоїмо кількість негативних прикладів

In [4]:
df = df.append(df[df['Sentiment']=='Negative'])

Розділимо дані на два сети:
- _тренувальний_ - 70%
- _тестовий_  - 30%

In [5]:
test_df = df.sample(frac=0.3)
train_df = df.drop(test_df.index)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, Perceptron
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import pymorphy2
import time

In [7]:
morph = pymorphy2.MorphAnalyzer(lang='uk')

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()  
        return lambda doc: ([morph.parse(w)[0].normal_form for w in analyzer(doc)])
    
class BigramCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(BigramCountVectorizer, self).build_analyzer()
        def bigram_analyzer(doc):
            tokens = analyzer(doc)
            return [' '.join(tokens[i:i+2]) for i in range(len(tokens)-1)]
        return bigram_analyzer

In [8]:
def evaluate_model(pipeline):
    start = time.time()
    
    text_clf = Pipeline(pipeline)
    text_clf = text_clf.fit(train_df.Comment, train_df.Sentiment)
    test_df['Predicted'] = text_clf.predict(test_df.Comment)
    
    print(f'Execution time: {time.time()-start:.1f}s')
    print(classification_report(test_df.Sentiment, test_df.Predicted))

#### Models evaluation

##### 1.1. Naive Bayes без нормалізації словоформ:

In [9]:
evaluate_model([('vect', CountVectorizer()),
                ('clf', MultinomialNB(fit_prior=False))])

Execution time: 0.3s
              precision    recall  f1-score   support

    Negative       0.81      0.42      0.55       556
    Positive       0.82      0.96      0.89      1520

   micro avg       0.82      0.82      0.82      2076
   macro avg       0.81      0.69      0.72      2076
weighted avg       0.82      0.82      0.80      2076



##### 1.2. Naive Bayes + нормалізація словоформ:

In [10]:
evaluate_model([('vect', StemmedCountVectorizer()),
                ('clf', MultinomialNB(fit_prior=False))])

Execution time: 23.8s
              precision    recall  f1-score   support

    Negative       0.76      0.52      0.62       556
    Positive       0.84      0.94      0.89      1520

   micro avg       0.83      0.83      0.83      2076
   macro avg       0.80      0.73      0.75      2076
weighted avg       0.82      0.83      0.82      2076



##### 1.3. Naive Bayes + біграми:

In [11]:
evaluate_model([('vect', BigramCountVectorizer()),
                ('clf', MultinomialNB(fit_prior=False))])

Execution time: 0.7s
              precision    recall  f1-score   support

    Negative       0.73      0.35      0.47       556
    Positive       0.80      0.95      0.87      1520

   micro avg       0.79      0.79      0.79      2076
   macro avg       0.77      0.65      0.67      2076
weighted avg       0.78      0.79      0.76      2076



##### 2.1. SVM:

In [17]:
evaluate_model([('vect', CountVectorizer()),
                ('clf-svm', SGDClassifier(max_iter=5, tol=1e-3))])

Execution time: 0.3s
              precision    recall  f1-score   support

    Negative       0.75      0.47      0.58       556
    Positive       0.83      0.94      0.88      1520

   micro avg       0.82      0.82      0.82      2076
   macro avg       0.79      0.71      0.73      2076
weighted avg       0.81      0.82      0.80      2076



##### 2.2. SVM + нормалізація словоформ:

In [13]:
evaluate_model([('vect', StemmedCountVectorizer()),
                ('clf-svm', SGDClassifier(max_iter=5, tol=1e-5))])

Execution time: 24.0s
              precision    recall  f1-score   support

    Negative       0.76      0.50      0.60       556
    Positive       0.84      0.94      0.89      1520

   micro avg       0.82      0.82      0.82      2076
   macro avg       0.80      0.72      0.74      2076
weighted avg       0.82      0.82      0.81      2076



##### 3. Logistic regression:

In [14]:
evaluate_model([('vect', CountVectorizer()),
                ('clf-lr', LogisticRegression(solver='liblinear'))])

Execution time: 0.3s
              precision    recall  f1-score   support

    Negative       0.84      0.37      0.51       556
    Positive       0.81      0.97      0.88      1520

   micro avg       0.81      0.81      0.81      2076
   macro avg       0.82      0.67      0.70      2076
weighted avg       0.82      0.81      0.78      2076



##### 4. Perceptron:

In [15]:
evaluate_model([('vect', CountVectorizer()),
                ('clf-lr', Perceptron(max_iter=5, tol=1e-5))])

Execution time: 0.3s
              precision    recall  f1-score   support

    Negative       0.74      0.45      0.56       556
    Positive       0.82      0.94      0.88      1520

   micro avg       0.81      0.81      0.81      2076
   macro avg       0.78      0.70      0.72      2076
weighted avg       0.80      0.81      0.79      2076



#### Спостереження
1. Усі лінійні класифікатори мають схожі результати
2. Нормалізація слів трішки покращує результати
3. Застосування tf-idf погіршує якість класифікації цього датасету
4. Модель Bag-of-words є занадто простою для гарної класифікаці негативних відгуків.
Наприклад: _"гарно працює"_, _"не працює"_, _"не ламається"_ неможливо розрізнити лише за наявністю певних слів у тексті.

#### References
1. [Machine Learning, NLP: Text Classification using scikit-learn, python and NLTK.](https://towardsdatascience.com/machine-learning-nlp-text-classification-using-scikit-learn-python-and-nltk-c52b92a7c73a)